In [118]:
import gurobipy as gp
from gurobipy import GRB


In [119]:
#Constants we are going to use
total_weeks = 2
hours_in_week = 24 * 7
hours_in_day = 24
Dt = 12 #hours


In [120]:
# Model sets
tasks = ['T11', 'T12', 'T21', 'T22', 'T31', 'T32', 'T41', 'T42', 'T43', 'T51', 'T52']
units = ['U1', 'U2', 'U3', 'U4', 'U5']
time_points = [i for i in range(int(total_weeks * hours_in_week / Dt) + 1)]
time_periods = [i for i in range(1, int(total_weeks * hours_in_week / Dt) + 1)]
materials = ['RM1', 'RM2', 'INT1', 'INT2', 'INT3', 'INT4', 'A', 'B1', 'B2', 'C1', 'C2']


In [121]:
#Model parameters
capacities = [80, 80, 5, 5, 5, 5, 10, 10, 10, 10, 10] #materials
production_costs = [1.0, 1.0, 0.5, 0.5, 1.5, 1.5, 1.0, 0.5, 0.5, 1.0, 0.5] #tasks
storage_costs = [0, 0, 0, 0, 0, 0, 1.5, 1.4, 1.8, 1.2, 2.0] #materials
daily_production_rate = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1] #tasks
production_rate = [rate * Dt / hours_in_day for rate in daily_production_rate]
onhand_inventory_list = [ 80, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0]

key_value_pairs_storage_cost = zip(materials, storage_costs)
key_value_pairs_production_cost = zip(tasks, production_costs)
key_value_pairs_capacity = zip(materials, capacities)
key_value_pairs_production_rate = zip(tasks, production_rate)
key_value_pairs_onhand_inventory = zip(materials, onhand_inventory_list)

material_storage_cost = dict(key_value_pairs_storage_cost)
task_production_cost = dict(key_value_pairs_production_cost)
capacity = dict(key_value_pairs_capacity)
rate = dict(key_value_pairs_production_rate)
onhand_inventory = dict(key_value_pairs_onhand_inventory)

unit_can_process_tasks = {
  'U1': ['T11', 'T12'],
  'U2': ['T21', 'T22'],
  'U3': ['T31', 'T32'],
  'U4': ['T41', 'T42', 'T43'],
  'U5': ['T51', 'T52']
}

material_produced_by_tasks = {
  'RM1': [],
  'RM2': [],
  'INT1': ['T11', 'T21'],
  'INT2': ['T12', 'T22'],
  'INT3': ['T31'],
  'INT4': ['T32'],
  'A': ['T41'],
  'B1': ['T42'],
  'B2': ['T51'],
  'C1': ['T43'],
  'C2': ['T52']
}

material_consumed_by_tasks = {
  'RM1': ['T11', 'T21'],
  'RM2': ['T12', 'T22'],
  'INT1': ['T31', 'T41'],
  'INT2': ['T32'],
  'INT3': ['T42', 'T51'],
  'INT4': ['T43', 'T52'],
  'A': [],
  'B1': [],
  'B2': [],
  'C1': [],
  'C2': []
}

net_delivery = {}
for k in materials:
  for n in time_points:
    net_delivery[(k,n)] = 0

net_delivery['A', hours_in_week / Dt] = -3
net_delivery['B1', hours_in_week / Dt] = -2.5
net_delivery['B2', hours_in_week / Dt] = -3.5
net_delivery['C1', hours_in_week / Dt] = -2.5
net_delivery['C2', hours_in_week / Dt] = -2.5
net_delivery['A', 2 * (hours_in_week / Dt)] = -3.5
net_delivery['B1', 2 * (hours_in_week / Dt)] = -3
net_delivery['B2', 2 * (hours_in_week / Dt)] = -3
net_delivery['C1', 2 * (hours_in_week / Dt)] = -2
net_delivery['C2', 2 * (hours_in_week / Dt)] = -3.5

print(net_delivery)

{('RM1', 0): 0, ('RM1', 1): 0, ('RM1', 2): 0, ('RM1', 3): 0, ('RM1', 4): 0, ('RM1', 5): 0, ('RM1', 6): 0, ('RM1', 7): 0, ('RM1', 8): 0, ('RM1', 9): 0, ('RM1', 10): 0, ('RM1', 11): 0, ('RM1', 12): 0, ('RM1', 13): 0, ('RM1', 14): 0, ('RM1', 15): 0, ('RM1', 16): 0, ('RM1', 17): 0, ('RM1', 18): 0, ('RM1', 19): 0, ('RM1', 20): 0, ('RM1', 21): 0, ('RM1', 22): 0, ('RM1', 23): 0, ('RM1', 24): 0, ('RM1', 25): 0, ('RM1', 26): 0, ('RM1', 27): 0, ('RM1', 28): 0, ('RM2', 0): 0, ('RM2', 1): 0, ('RM2', 2): 0, ('RM2', 3): 0, ('RM2', 4): 0, ('RM2', 5): 0, ('RM2', 6): 0, ('RM2', 7): 0, ('RM2', 8): 0, ('RM2', 9): 0, ('RM2', 10): 0, ('RM2', 11): 0, ('RM2', 12): 0, ('RM2', 13): 0, ('RM2', 14): 0, ('RM2', 15): 0, ('RM2', 16): 0, ('RM2', 17): 0, ('RM2', 18): 0, ('RM2', 19): 0, ('RM2', 20): 0, ('RM2', 21): 0, ('RM2', 22): 0, ('RM2', 23): 0, ('RM2', 24): 0, ('RM2', 25): 0, ('RM2', 26): 0, ('RM2', 27): 0, ('RM2', 28): 0, ('INT1', 0): 0, ('INT1', 1): 0, ('INT1', 2): 0, ('INT1', 3): 0, ('INT1', 4): 0, ('INT1', 5)

In [122]:
m = gp.Model("production-planning-simple")

In [123]:
#Define model variables
w = m.addVars(tasks, time_periods, vtype=GRB.BINARY, name="w")
w_s = m.addVars(units, time_periods, vtype=GRB.BINARY, name="w_s")

s = m.addVars(materials, time_points[1:], vtype=GRB.CONTINUOUS, lb=0, name="s") # MAYBE ITS NOT AN INTEGER

In [124]:
#Define objective function
obj = sum( material_storage_cost[k] * Dt / hours_in_day * s[k,n] for k in materials for n in time_points[1:]) + \
      sum(task_production_cost[i] * rate[i] * w[i,n] for n in time_periods for i in tasks)

m.setObjective(obj, GRB.MINIMIZE)

In [125]:
#Define constraints
capacity_constraint = m.addConstrs(s[k,n] <= capacity[k] for k in materials for n in time_points[1:])

assignment = m.addConstrs(sum(w[i,n] for i in unit_can_process_tasks[j]) + w_s[j,n] == 1 \
  for j in units for n in time_periods)

initial_balance = m.addConstrs(s[k,1] == onhand_inventory[k] + net_delivery[k,1] \
  + sum(rate[i] * w[i,1] for i in material_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,1] for i in material_consumed_by_tasks[k]) \
                             for k in materials )
                             
balance = m.addConstrs(s[k,n] == s[k,n-1] + net_delivery[k,n] \
  + sum(rate[i] * w[i,n] for i in material_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,n] for i in material_consumed_by_tasks[k]) \
  for k in materials for n in time_points[2:] )
            

In [126]:
m.write('test.lp' )

In [127]:
m.optimize()

Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 756 rows, 756 columns and 1977 nonzeros
Model fingerprint: 0x30c00e0b
Variable types: 308 continuous, 448 integer (448 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+00]
  Objective range  [3e-01, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 8e+01]
Presolve removed 489 rows and 326 columns
Presolve time: 0.01s
Presolved: 267 rows, 430 columns, 1052 nonzeros
Variable types: 0 continuous, 430 integer (308 binary)

Root relaxation: objective 2.024250e+02, 245 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  202.42500    0    3          -  202.42500 